Install the required packages

---



In [ ]:
pip install scenedetect

In [ ]:
pip install deepface

In [ ]:
pip install face_recognition

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import the required packages

In [ ]:
#from __future__ import print_function
import os
import cv2
from deepface import DeepFace
#from deepface.extendedmodels import Race,Gender

import face_recognition
import pandas as pd

from tqdm import tqdm
import pickle
import json

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from keras.models import load_model
from keras.preprocessing import image

from deepface.basemodels import VGGFace
import os
from pathlib import Path
import gdown
import numpy as np


In [ ]:
angle_model = load_model('/content/drive/MyDrive/models/TEST_ANGLE.h5',compile = True)
shot_model = load_model('/content/drive/MyDrive/models/Shot Class model.h5',compile = True)

Run scenedetect on the desired video file or video files

In [ ]:
!scenedetect -i /content/for_a_few_dollars_more.mp4 -o output_dir detect-content list-scenes save-images

[PySceneDetect] PySceneDetect v0.5.5
[PySceneDetect] Output directory set:
  /content/output_dir
[PySceneDetect] Loaded 1 video, framerate: 29.97 FPS, resolution: 854 x 480
[PySceneDetect] Downscale factor set to 3, effective resolution: 284 x 160
[PySceneDetect] Scene list CSV file name format:
  $VIDEO_NAME-Scenes.csv
[PySceneDetect] Image output format set: JPEG [Quality: 95]
[PySceneDetect] Detecting scenes...
100% 12163/12163 [00:29<00:00, 407.04frames/s]
[PySceneDetect] Processed 12163 frames in 29.9 seconds (average 406.95 FPS).
[PySceneDetect] Detected 95 scenes, average shot length 4.3 seconds.
[PySceneDetect] Writing scene list to CSV file:
  /content/output_dir/for_a_few_dollars_more-Scenes.csv
[PySceneDetect] Scene List:
-----------------------------------------------------------------------
 | Scene # | Start Frame |  Start Time  |  End Frame  |   End Time   |
-----------------------------------------------------------------------
 |      1  |           0 | 00:00:00.000 | 

Create a list of the output of Scenedetect

In [ ]:

files = os.listdir("/content/output_dir/content/output_dir")
#files = files[:-1]
files = files[0::3]
print(len(files))

96


Define our prediction and classification functions

The below four functions are copied from the Deepface github repository
Normally I would use the Deepface API and get this done with 3 lines of code.
However Deepface experienced heavy traffic that does not let users download the Race and Gender model weights when You run the function.

This solution is more complicated then just using the API, however it is less resource intesive as You only have to load the weights once when You run the for loop on multiple images. 

For the four functions below all credit goes to the Deepface dev team, I only made minor changes

In [ ]:
import shutil
shutil.unpack_archive("/content/drive/MyDrive/models/race_model_single_batch.zip", "/content/modellek/")
shutil.unpack_archive("/content/output.zip", "/content/output_dir/")

In [ ]:
def loadraceModel():
	
	model = VGGFace.baseModel()
	
	#--------------------------
	
	classes = 6
	base_model_output = Sequential()
	base_model_output = Convolution2D(classes, (1, 1), name='predictions')(model.layers[-4].output)
	base_model_output = Flatten()(base_model_output)
	base_model_output = Activation('softmax')(base_model_output)
	
	#--------------------------

	race_model = Model(inputs=model.input, outputs=base_model_output)
	
	#--------------------------
	
	#load weights
	

	race_model.load_weights('/content/modellek/race_model_single_batch.h5')
	
	return race_model
    
    
def loadgenderModel():
	
	model = VGGFace.baseModel()
	
	#--------------------------
	
	classes = 2
	base_model_output = Sequential()
	base_model_output = Convolution2D(classes, (1, 1), name='predictions')(model.layers[-4].output)
	base_model_output = Flatten()(base_model_output)
	base_model_output = Activation('softmax')(base_model_output)
	
	#--------------------------

	gender_model = Model(inputs=model.input, outputs=base_model_output)
	
	gender_model.load_weights('/content/drive/MyDrive/models/gender_model_weights.h5')
	
	return gender_model


def build_model(model_name):

	"""
	This function builds a deepface model
	Parameters:
		model_name (string): face recognition or facial attribute model
			VGG-Face, Facenet, OpenFace, DeepFace, DeepID for face recognition
			Age, Gender, Emotion, Race for facial attributes
	Returns:
		built deepface model
	"""

	models = {
		'VGG-Face': VGGFace.loadModel,

		#'Age': Age.loadModel,
		'Gender': gender,
		'Race': race
	}

	model = models.get(model_name)

In [ ]:
gender = loadgenderModel()
race = loadraceModel()

In [ ]:
from deepface.commons import functions, realtime, distance as dst
def analyze(img_path, actions = ['emotion', 'age', 'gender', 'race'] , models = {}, enforce_detection = True, detector_backend = 'mtcnn'):


	img_paths, bulkProcess = functions.initialize_input(img_path)
	functions.initialize_detector(detector_backend = detector_backend)

	#---------------------------------

	built_models = list(models.keys())

	#---------------------------------

	#pre-trained models passed but it doesn't exist in actions
	if len(built_models) > 0:
		if 'emotion' in built_models and 'emotion' not in actions:
			actions.append('emotion')

		if 'age' in built_models and 'age' not in actions:
			actions.append('age')

		if 'gender' in built_models and 'gender' not in actions:
			actions.append('gender')

		if 'race' in built_models and 'race' not in actions:
			actions.append('race')

	#---------------------------------

	if 'emotion' in actions and 'emotion' not in built_models:
		models['emotion'] = build_model('Emotion')

	if 'age' in actions and 'age' not in built_models:
		models['age'] = build_model('Age')

	if 'gender' in actions and 'gender' not in built_models:
		models['gender'] = build_model('Gender')

	if 'race' in actions and 'race' not in built_models:
		models['race'] = build_model('Race')

	#---------------------------------

	resp_objects = []

	disable_option = False if len(img_paths) > 1 else True

	global_pbar = tqdm(range(0,len(img_paths)), desc='Analyzing', disable = disable_option)

	for j in global_pbar:
		img_path = img_paths[j]

		resp_obj = {}

		disable_option = False if len(actions) > 1 else True

		pbar = tqdm(range(0,len(actions)), desc='Finding actions', disable = disable_option)

		img_224 = None # Set to prevent re-detection

		region = [] # x, y, w, h of the detected face region
		region_labels = ['x', 'y', 'w', 'h']

		#facial attribute analysis
		for index in pbar:
			action = actions[index]
			pbar.set_description("Action: %s" % (action))

			if action == 'emotion':
				emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
				img, region = functions.preprocess_face(img = img_path, target_size = (48, 48), grayscale = True, enforce_detection = enforce_detection, detector_backend = detector_backend, return_region = True)

				resp_obj["region"] = {}

				for i, parameter in enumerate(region_labels):
					resp_obj["region"][parameter] = region[i]

				emotion_predictions = models['emotion'].predict(img)[0,:]

				sum_of_predictions = emotion_predictions.sum()

				resp_obj["emotion"] = {}

				for i in range(0, len(emotion_labels)):
					emotion_label = emotion_labels[i]
					emotion_prediction = 100 * emotion_predictions[i] / sum_of_predictions
					resp_obj["emotion"][emotion_label] = emotion_prediction

				resp_obj["dominant_emotion"] = emotion_labels[np.argmax(emotion_predictions)]

			elif action == 'age':
				if img_224 is None:
					img_224, region = functions.preprocess_face(img = img_path, target_size = (224, 224), grayscale = False, enforce_detection = enforce_detection, detector_backend = detector_backend, return_region = True)

				resp_obj["region"] = {}

				for i, parameter in enumerate(region_labels):
					resp_obj["region"][parameter] = region[i]

				age_predictions = models['age'].predict(img_224)[0,:]
				apparent_age = Age.findApparentAge(age_predictions)

				resp_obj["age"] = int(apparent_age)

			elif action == 'gender':
				if img_224 is None:
					img_224, region = functions.preprocess_face(img = img_path, target_size = (224, 224), grayscale = False, enforce_detection = enforce_detection, detector_backend = detector_backend, return_region = True)

				resp_obj["region"] = {}

				for i, parameter in enumerate(region_labels):
					resp_obj["region"][parameter] = region[i]

				gender_prediction = models['gender'].predict(img_224)[0,:]

				if np.argmax(gender_prediction) == 0:
					gender = "Woman"
				elif np.argmax(gender_prediction) == 1:
					gender = "Man"

				resp_obj["gender"] = gender

			elif action == 'race':
				if img_224 is None:
					img_224, region = functions.preprocess_face(img = img_path, target_size = (224, 224), grayscale = False, enforce_detection = enforce_detection, detector_backend = detector_backend, return_region = True) #just emotion model expects grayscale images
				race_predictions = models['race'].predict(img_224)[0,:]
				race_labels = ['asian', 'indian', 'black', 'white', 'middle eastern', 'latino hispanic']

				resp_obj["region"] = {}

				for i, parameter in enumerate(region_labels):
					resp_obj["region"][parameter] = region[i]

				sum_of_predictions = race_predictions.sum()

				resp_obj["race"] = {}
				for i in range(0, len(race_labels)):
					race_label = race_labels[i]
					race_prediction = 100 * race_predictions[i] / sum_of_predictions
					resp_obj["race"][race_label] = race_prediction

				resp_obj["dominant_race"] = race_labels[np.argmax(race_predictions)]

		#---------------------------------

		if bulkProcess == True:
			resp_objects.append(resp_obj)
		else:
			return resp_obj

	if bulkProcess == True:

		resp_obj = {}

		for i in range(0, len(resp_objects)):
			resp_item = resp_objects[i]
			resp_obj["instance_%d" % (i+1)] = resp_item

		return resp_obj

In [ ]:
class Predict():
# check_face retuns 0 if there is no face, or the number of faces on an image
    def check_face(file):
        img = cv2.imread(file,1)
        #image = np.array(img)
        image = face_recognition.load_image_file(file)
# run the face recognition on an image
        face_locations = face_recognition.face_locations(image, number_of_times_to_upsample=1, model="cnn")

        if len(face_locations) > 0:
#If there is a face detected create a list of point using face landmarks. 
#These will be used when calculating the contrast ratio of the face.
# points and points2 are the point of interest on the face. ROI and ROI2 are the rectangle we draw from these points.
            lista = face_recognition.face_landmarks(img,face_locations = face_locations)
            points = [lista[0]['chin'][0],lista[0]['nose_bridge'][-1]]
            points2 = [lista[0]['nose_bridge'][0],lista[0]['chin'][-4]]
            ROI = img[points[0][1]:points[1][1], points[0][0]:points[1][0]]
            ROI2 = img[points2[0][1]:points2[1][1], points2[0][0]:points2[1][0]]
            return len(face_locations),ROI,ROI2
        else:
            return 0,0,0



    def analyze(image,category1,category2,model1,model2):
        img = cv2.imread(image,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img,(320,180))
        new_img = img.reshape(-1,320,180,1)
        pred1 = model1.predict_classes(new_img)
        pred2 = model2.predict_classes(new_img)
        class1 = category1[int(pred1)]
        class2 = category2[int(pred2)]
        return class1,class2
        


In [ ]:
# This function takes the ROI rectangles above and calculates the brightness value of each pixel,
# Then gives us the average brightness of that territory. If we do this for both side of the face we get the contrast ratio


from numpy.linalg import norm
#https://stackoverflow.com/questions/14243472/estimate-brightness-of-an-image-opencv

def brightness(img):

    if type(img) == int:
        return str(0)
    elif len(img.shape) == 3:
        # Colored RGB or BGR (*Do Not* use HSV images with this function)
        # create brightness with euclidean norm
        return str(np.average(norm(img, axis=2)) / np.sqrt(3))
    else:
        # Grayscale
        return str(np.average(img))





Create empty lists where the loop will put the values to

In [ ]:
filename_list = []
face_list = []
race_list = []
gender_list = []
brightness_list = []
angle_return = []
shot_return = []
shot_class = ['CU','MC','ML','Long']
angle_class = ['High','Med','Low']

In [ ]:
%cd /content/output_dir/content/output_dir

/content/output_dir/content/output_dir


In [ ]:
!pwd

/content/output_dir/content/output_dir


In [ ]:
models = {}
models["gender"]=loadgenderModel()
models["race"] = loadraceModel()

Run a loop on the files

In [ ]:
#os.chdir('/content/output_dir')
for file in files[1:30]:
    print(file)
    face,ROI,ROI2 = Predict.check_face(file)
    face_list.append(face)

    Side1 = brightness(ROI)
    Side2 = brightness(ROI2)
    brightness_list.append(f'{Side1[:4]} : {Side2[:4]}')
    
    race_class = analyze(file,actions=['race'],models=models, enforce_detection=False)
    race_data = race_class.get("dominant_race")
    race_list.append(race_data)
    
    gender_class =  analyze(file,actions=['gender'],models=models, enforce_detection=False)
    gender_data = gender_class.get("gender")
    gender_list.append(gender_data)
    
    filename_list.append(file)

    img = cv2.imread(file,cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img,(320,180))
    new_img = img.reshape(-1,320,180,1)
    pred1 = angle_model.predict_classes(new_img)
    pred2 = shot_model.predict_classes(new_img)
    class1 = angle_class[int(pred1)]
    class2 = shot_class[int(pred2)]
    shot_return.append(class2)
    angle_return.append(class1)

    #shot,angle = Predict.analyze(file,shot_class,angle_class,shot_model,angle_model)
    #angle_return.append(angle)
    #shot_return.append(shot)

for_a_few_dollars_more-Scene-009-03.jpg



Action: gender: 100%|██████████| 2/2 [00:03<00:00,  1.61s/it]

Action: race: 100%|██████████| 2/2 [00:01<00:00,  1.76it/s]


for_a_few_dollars_more-Scene-069-02.jpg



Action: gender: 100%|██████████| 2/2 [00:01<00:00,  1.84it/s]

Action: race: 100%|██████████| 2/2 [00:01<00:00,  1.99it/s]


for_a_few_dollars_more-Scene-082-01.jpg


...


Action: gender: 100%|██████████| 2/2 [00:01<00:00,  1.14it/s]

Action: race: 100%|██████████| 2/2 [00:01<00:00,  1.24it/s]


for_a_few_dollars_more-Scene-026-02.jpg



Action: gender: 100%|██████████| 2/2 [00:01<00:00,  1.05it/s]

Action: race: 100%|██████████| 2/2 [00:01<00:00,  1.48it/s]


for_a_few_dollars_more-Scene-060-03.jpg



Action: gender: 100%|██████████| 2/2 [00:01<00:00,  1.16it/s]

Action: race: 100%|██████████| 2/2 [00:01<00:00,  1.48it/s]


Create a dataframe based on the received values

In [ ]:
df = pd.DataFrame(list(zip(filename_list,race_list,gender_list,face_list,brightness_list,angle_return,shot_return)), columns = ['Filename','Race','Gender','Face','Brightness','Camera Angle','Shot Type'])
df

,Filename,Race,Gender,Face,Brightness,Camera Angle,Shot Type
0,for_a_few_dollars_more-Scene-009-03.jpg,latino hispanic,Man,1,77.0 : 73.3,Med,MC
1,for_a_few_dollars_more-Scene-069-02.jpg,white,Man,0,0 : 0,Med,CU
2,for_a_few_dollars_more-Scene-082-01.jpg,white,Man,1,48.2 : 82.2,Med,CU
3,for_a_few_dollars_more-Scene-088-01.jpg,latino hispanic,Man,1,101. : 44.1,Med,Long
4,for_a_few_dollars_more-Scene-072-01.jpg,white,Man,1,46.4 : 77.5,Med,CU
5,for_a_few_dollars_more-Scene-032-01.jpg,white,Man,1,55.4 : 80.4,Med,CU
6,for_a_few_dollars_more-Scene-019-01.jpg,indian,Man,1,72.8 : 86.0,Med,ML
7,for_a_few_dollars_more-Scene-008-01.jpg,indian,Man,1,63.9 : 83.6,Low,CU
8,for_a_few_dollars_more-Scene-057-01.jpg,asian,Man,1,52.8 : 87.4,Med,CU
9,for_a_few_dollars_more-Scene-093-02.jpg,middle eastern,Man,1,84.9 : 61.3,Med,CU


In [ ]:
df.to_csv('Film_shots.csv')

,Filename,Race,Gender,Face,Brightness,Camera Angle,Shot Type
99,for_a_few_dollars_more-Scene-012-03.jpg,white,Man,0,0 : 0,Med,CU
100,for_a_few_dollars_more-Scene-025-03.jpg,white,Man,1,59.9 : 65.8,Med,CU
101,for_a_few_dollars_more-Scene-012-01.jpg,white,Man,0,0 : 0,Med,CU
102,for_a_few_dollars_more-Scene-095-02.jpg,white,Man,1,97.9 : 72.5,Med,CU
103,for_a_few_dollars_more-Scene-090-01.jpg,white,Man,1,116. : 80.0,Med,CU


In [ ]:
!zip -r /content/output.zip /content/output_dir

  adding: content/output_dir/ (stored 0%)
  adding: content/output_dir/for_a_few_dollars_more-Scene-089-01.jpg (deflated 3%)
  adding: content/output_dir/for_a_few_dollars_more-Scene-014-01.jpg (deflated 4%)
  adding: content/output_dir/for_a_few_dollars_more-Scene-007-03.jpg (deflated 5%)
  adding: content/output_dir/for_a_few_dollars_more-Scene-015-03.jpg (deflated 2%)
  adding: content/output_dir/for_a_few_dollars_more-Scene-058-02.jpg (deflated 4%)
  adding: content/output_dir/for_a_few_dollars_more-Scene-037-02.jpg (deflated 7%)
  adding: content/output_dir/for_a_few_dollars_more-Scene-009-03.jpg (deflated 4%)
  adding: content/output_dir/for_a_few_dollars_more-Scene-083-02.jpg (deflated 3%)
  adding: content/output_dir/for_a_few_dollars_more-Scene-016-02.jpg (deflated 6%)
  adding: content/output_dir/for_a_few_dollars_more-Scene-042-03.jpg (deflated 3%)
  adding: content/output_dir/for_a_few_dollars_more-Scene-046-02.jpg (deflated 6%)
  adding: content/output_dir/for_a_few_dollar